In [1]:
aim = 'zh_test.en'
src_lang = 'en'
tgt_lang = 'zh-cn'

In [4]:
bscnt = 0
bscnt2 = 0
with open(aim + '.src.tok_by_spacy', encoding='utf-8') as fsrc, open(aim + '.tgt', encoding='utf-8') as ftgt, \
     open(aim, 'w', encoding='utf-8') as fout:
    for i, (src, tgt) in enumerate(zip(fsrc, ftgt)):
        src = src.strip()
        tgt = tgt.strip()
        
        if len(src + '\n' + tgt) > 1500:
            src = ' '.join(src.split()[:50])
            tgt = ' '.join(tgt.split()[:50])
            bscnt2 += 1
        if src == '':
            src = '<bs>'
            bscnt += 1
        if tgt == '':
            tgt = '<bs>'
            bscnt += 1
        fout.write(src + '\t' + tgt + '\n')
print(bscnt)
print(bscnt2)

0
0


In [ ]:
# coding: utf-8
# 支持中文到英文的翻译，设置zh和en变量之后直接运行即可
import re
import time
import nltk
import logging
from googletrans import Translator

logging.basicConfig(level = logging.INFO, format = '%(asctime)s - %(name)s - %(levelname)s - %(message)s')


# read these files
# 文件每行的格式为：\t分隔的两个中文字符串（分词、未分词均可）
zh = aim

# write to these files
# 文件每行的格式为：\t分隔的两个分词后的英文字符串
en = aim + f'.{tgt_lang}'

SP = '\n'
SP_LEN = len(SP)
# 一次传输的字符（BUF）长度不能至多大于2000，保守取1500
MAX_BUF = 1500

translator = Translator(service_urls=['translate.google.cn'])


def sentencetranslate(line, src=src_lang, dest=tgt_lang):
    try:
        line = line.strip()
        text = translator.translate(line, src=src, dest=dest).text
        return text
    except Exception as e:
        logging.info('翻译报错')
        return 'blank'


def transalate_file2file_zh2en(fr, fw):
    has_blank = False
    f1 = open(fr, 'r', encoding='utf-8')
    lines = f1.readlines()
    f1.close()
    i = 0
    max_i = len(lines)
    logging.info('共有{}行'.format(max_i))
    q_r = []
    for line in lines:
        q_s, r_s = line.strip().split('\t')#replace(' ', '').split('\t')
        q_r.append((q_s, r_s))
    with open(fw, 'w', encoding='utf-8') as f2:
        logging.info('Begin...')
        buf = ''
        lines_cnt_in_buf = 0
        while i <= max_i:
            if i == max_i or len(buf + q_r[i][0] + SP + q_r[i][1]) > MAX_BUF:
                # 如果buf加上下面一行超过MAX_BUF的话，则翻译buf的内容
                # 如果已经遍历完了（i为max_i），buf中的长度一定不超过MAX_BUF，直接翻译buf中的内容，并结束循环
                buf = buf[:-SP_LEN] # 去掉最后一个SP
                t_buf = sentencetranslate(buf)
                if t_buf == 'blank':
                    has_blank = True
                    for j in range(lines_cnt_in_buf):
                        f2.write('blank\n')
                else:
                    t_buf = re.split(SP, t_buf)
                    assert len(t_buf) == 2 * lines_cnt_in_buf, 'Missing sentences'
                    for j in range(lines_cnt_in_buf):
                        q_t = t_buf[2*j].strip()
                        r_t = t_buf[2*j+1].strip()
#                         q_t = (' '.join(nltk.word_tokenize(q_t))).lower()
#                         r_t = (' '.join(nltk.word_tokenize(r_t))).lower()
                        f2.write(q_t + '\t' + r_t + '\n')
                logging.info('Processing {} line'.format(i+1))
                if i == max_i:
                    break
                buf = ''
                lines_cnt_in_buf = 0
                time.sleep(2) # 休息1s意思一下。可能即使不主动sleep，翻译之间的处理也有一定的时间间隔
            else:
                buf += q_r[i][0] + SP + q_r[i][1] + SP
                lines_cnt_in_buf += 1
                i += 1
    return has_blank

def check_valid(fr):
    # 每行包括两句非空的话
    # S1+SP+S2的长度要小于等于MAX_BUF
    with open(fr, 'r', encoding='utf-8') as f1:
        for i, line in enumerate(f1):
            q_s, r_s = line.strip().split('\t')
            assert q_s != '', 'q_s不能为空（行号{}）'.format(i+1)
            assert r_s != '', 'r_s不能为空（行号{}）'.format(i+1)
            assert len(q_s + SP + r_s) <= MAX_BUF, '这一行（行号{}）过于长，看着办吧'.format(i+1)


def process_blank_zh2en(zh, en):
    fzh = open(zh, 'r', encoding='utf-8')
    fen = open(en, 'r', encoding='utf-8')
    zh_lines = fzh.readlines()
    en_lines = fen.readlines()
    fzh.close()
    fen.close()
    assert len(zh_lines) == len(en_lines), 'process_blank: assert len(zh_lines) == len(en_lines)'
    f_tmp = open('tmp', 'w', encoding='utf-8')
    for zh_s, en_s in zip(zh_lines, en_lines):
        zh_s = zh_s.strip()
        en_s = en_s.strip()
        if en_s == 'blank':
            q_s, r_s = zh_s.replace(' ', '').split('\t')
            q_t = sentencetranslate(q_s)
#             q_t = (' '.join(nltk.word_tokenize(q_t))).lower()
            r_t = sentencetranslate(r_s)
#             r_t = (' '.join(nltk.word_tokenize(r_t))).lower()
            f_tmp.write(q_t + '\t' + r_t + '\n')
        else:
            f_tmp.write(en_s + '\n')
    f_tmp.close()
    f_tmp = open('tmp', 'r', encoding='utf-8')
    fen = open(en, 'w', encoding='utf-8')
    for line in f_tmp:
        fen.write(line)
    f_tmp.close()
    fen.close()


logging.info('检查数据的合法性')
check_valid(zh)
logging.info('数据合法')
logging.info("开始s翻译")
has_blank = transalate_file2file_zh2en(zh, en)
if has_blank:
    # 说明有些句子没有翻译好，后处理一下
    logging.info('翻译blank行')
    process_blank_zh2en(zh, en)
logging.info("结束翻译")

2019-12-05 17:57:46,462 - root - INFO - 检查数据的合法性
2019-12-05 17:57:46,479 - root - INFO - 数据合法
2019-12-05 17:57:46,481 - root - INFO - 开始s翻译
2019-12-05 17:57:46,491 - root - INFO - 共有10000行
2019-12-05 17:57:46,500 - root - INFO - Begin...
2019-12-05 17:57:46,966 - root - INFO - Processing 15 line
2019-12-05 17:57:49,172 - root - INFO - Processing 28 line


In [5]:
from tqdm import tqdm
aim='zh_test.en'
tgt_lang='zh-cn'

with open(f'{aim}.{tgt_lang}', encoding='utf-8') as ft, \
     open(f'{aim}.{tgt_lang}.tgt', 'w', encoding='utf-8') as ftr:
    for i, line in tqdm(enumerate(ft)):
        try:
            q, r = line.strip().split('\t')
        except:
            q, r = '', ''
        q += '\n'
        r += '\n'
        
        ftr.write(r)
    

10000it [00:00, 332530.27it/s]
